# Preparação dos Dados   

## 1 - Bibliotecas e Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import ttest_ind, mannwhitneyu
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.api as sm
import featuretools as ft
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle

df = pd.read_csv('df_EDA.csv')

## 2 - Tratamento de Missing Values

In [8]:
df.isna().sum()

Unnamed: 0           0
Series_Title         0
Released_Year        1
Released_Month     216
Released_Day       216
Certificate        101
Runtime              0
Genre                0
IMDB_Rating          0
Overview             0
Meta_score         156
Director             0
Star1                0
Star2                0
Star3                0
Star4                0
No_of_Votes          0
Gross              169
release_date       216
budget             112
ROI                331
Gross_log          169
No_of_Votes_log      0
budget_log         112
ROI_log            331
dtype: int64

### 2.1 - Released Year

Considerando que não teria como imputar o ano que o filme foi produzido, mas também que só tem um filme sem essa informação, vou inserir essa informação

In [24]:
df[df['Released_Year'].isna()]

,Unnamed: 0,Series_Title,Released_Year,Released_Month,Released_Day,Certificate,Runtime,Genre,IMDB_Rating,Overview,...,Star4,No_of_Votes,Gross,release_date,budget,ROI,Gross_log,No_of_Votes_log,budget_log,ROI_log
964,966,Apollo 13,NaN,6.0,30.0,U,140,"Adventure, Drama, History",7.6,NASA must devise a strategy to return Apollo 1...,...,Gary Sinise,269197,173837933.0,1995-06-30,52000000.0,2.343037,18.973634,12.503202,17.766754,1.20688


Apolo 13 é um filme de 1995, logo vou inserir esse valor

In [42]:
df.loc[df['Released_Year'].isna(), 'Released_Year'] = 1995

### 2.2 - Released Month e Released Day e Released Date

Por conta do alto valor de missings nessas variáveis, vou retiirá-las do modelo, assim evita de perder muita informação

In [43]:
df = df.drop(columns=['Released_Month', 'Released_Day', 'release_date'])

### 2.3 - Certificate

Para manter uma coerência nas classificações, vou utilizar a moda por gênero, assim gêneros com classificações mais severas como Terror, segurão essas linhas, e mais brandos como romances e familiares também terão uma lógica

In [44]:
df['Main_Genre'] = df['Genre'].str.split(',').str[0]
df['Certificate'] = df.groupby('Main_Genre', group_keys=False)['Certificate'] \
                      .apply(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Not Rated'))

### 2.4 - Meta_score

Existem duas opções claras: Imputar diretamente pela mediana, ou usar IMDB_rating como base, vou verificar a correlação (já vista no mapa de correlação na EDA)

In [28]:
corr = df.loc[df['Meta_score'].notna(), 'Meta_score'].corr(
    df.loc[df['Meta_score'].notna(), 'IMDB_Rating']
)
print(f"Correlação entre IMDB_Rating e Meta_score: {corr:.2f}")

Correlação entre IMDB_Rating e Meta_score: 0.27


Como a correlação é baixa, vou usar a mediana

In [45]:
df['Meta_score'] = df['Meta_score'].fillna(df['Meta_score'].median())

### 2.5 Gross, Budget e ROI (e seus logs)

In [30]:
corr_bg = df.loc[df['budget'].notna() & df['Gross'].notna(), 'budget'] \
           .corr(df.loc[df['budget'].notna() & df['Gross'].notna(), 'Gross'])

print(f"Correlação entre budget e Gross: {corr_bg:.2f}")

Correlação entre budget e Gross: 0.77


Levando em consideração a alta correlação, vou utilizá-los para realizar o imput onde for possível

In [46]:
mask_budget = df['budget'].isna() & df['Gross'].notna()
if mask_budget.any():
    model_budget = LinearRegression()
    train_data = df[df['budget'].notna() & df['Gross'].notna()]
    model_budget.fit(train_data[['Gross']], train_data['budget'])
    df.loc[mask_budget, 'budget'] = model_budget.predict(df.loc[mask_budget, ['Gross']])

mask_gross = df['Gross'].isna() & df['budget'].notna()
if mask_gross.any():
    model_gross = LinearRegression()
    train_data = df[df['Gross'].notna() & df['budget'].notna()]
    model_gross.fit(train_data[['budget']], train_data['Gross'])
    df.loc[mask_gross, 'Gross'] = model_gross.predict(df.loc[mask_gross, ['budget']])
    df.loc[mask_gross, 'Gross'] = model_gross.predict(df.loc[mask_gross, ['budget']])

In [47]:
df.isna().sum()

Unnamed: 0           0
Series_Title         0
Released_Year        0
Certificate          0
Runtime              0
Genre                0
IMDB_Rating          0
Overview             0
Meta_score           0
Director             0
Star1                0
Star2                0
Star3                0
Star4                0
No_of_Votes          0
Gross               35
budget              35
ROI                331
Gross_log          169
No_of_Votes_log      0
budget_log         112
ROI_log            331
Main_Genre           0
dtype: int64

dos 169 dados ausentes em Gross e 112 em Budget, sobraram apenas 36 em cada. Vou preencher o restante com a mediana e em seguida refazer as colunas de ROI e as transformações logarítimicas

In [48]:
df['budget'] = df['budget'].fillna(df['budget'].median())
df['Gross'] = df['Gross'].fillna(df['Gross'].median())

In [49]:
df['ROI'] = (df['Gross'] - df['budget']) / df['budget']

df['Gross_log'] = np.log1p(df['Gross'])
df['budget_log'] = np.log1p(df['budget'])
df['ROI_log'] = np.log1p(df['ROI'])

In [50]:
df.isna().sum()

Unnamed: 0         0
Series_Title       0
Released_Year      0
Certificate        0
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score         0
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross              0
budget             0
ROI                0
Gross_log          0
No_of_Votes_log    0
budget_log         0
ROI_log            0
Main_Genre         0
dtype: int64

In [51]:
df_split = df.copy()

## 3 - Separação em Treino e Teste (Split)

Considerando que a proposta é prever a nota do imdb, vou isolar ela e fazer o split na proporção 70-30 por ser um dataset pequeno.
O modelo será de classificação, não vou utilizar uma estratificação pois algumas notas são muito raras, não valendo a pena criar uma categoria de 'outros' apenas para elas, já que podem ser muito altas ou muito baixas

In [ ]:
df_split['IMDB_Rating'] = df_split['IMDB_Rating'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## 4 - Feature Engineering

Para a engenharia de variáveis, vou usar o feature tools, pacote que permite fazer o processo de forma automatizada, assim extraindo mais informações do dataset

In [ ]:
es = ft.EntitySet(id='movies')

es = es.add_dataframe(
    dataframe_name='movies_train',
    dataframe=X_train,
    index='index',  
)

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='movies_train',
    max_depth=2,   
    verbose=1
)

feature_matrix.head()

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\featuretools\entityset\entityset.py:1733: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python310\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


Built 16 features
Elapsed: 00:00 | Progress: 100%|██████████


,Unnamed: 0,Released_Year,Certificate,Runtime,Meta_score,No_of_Votes,Gross,budget,ROI,Gross_log,No_of_Votes_log,budget_log,ROI_log,Main_Genre,NUM_CHARACTERS(Overview),NUM_WORDS(Overview)
index,,,,,,,,,,,,,,,,
0,745,2013.0,UA,91,96.0,769145,2.740927e+08,105000000.0,1.610407,19.428977,13.553036,18.469471,0.959506,Drama,88,14
1,288,1967.0,A,127,92.0,161984,1.621777e+07,3200000.0,4.068054,16.601618,11.995259,14.978662,1.622957,Crime,93,18
2,167,1992.0,A,130,85.0,375935,1.011574e+08,14400000.0,6.024823,18.432189,12.837174,16.482739,1.949450,Drama,157,28
3,961,1997.0,A,134,52.0,131101,3.796699e+06,15000000.0,-0.746887,15.149643,11.783731,16.523561,-1.373918,Mystery,113,15
4,495,2007.0,U,87,79.0,174125,1.679594e+07,200000.0,82.979680,16.636648,12.067534,12.206078,4.430575,Drama,197,31


O feature tools me deu mais duas variáveis, número de palavras e caracteres na sinópse, agora vou aplicar a mesma coisa no teste

In [56]:
X_test['NUM_CHARACTERS(Overview)'] = X_test['Overview'].apply(len)
X_test['NUM_WORDS(Overview)'] = X_test['Overview'].apply(lambda x: len(x.split()))

In [59]:
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open('y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)